# 데이터베이스에서 가져오기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/hg-da/blob/main/Appendix-A.ipynb"><img src="https://jupyter.org/assets/share.png" width="61" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-da/blob/main/Appendix-A.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

## 파이썬에서 SQL 사용하기: SQLite

In [1]:
# 최신 sqlalchemy는 판다스에서 에러를 일으킵니다. 1.4.* 버전을 사용해 주세요. (https://github.com/pandas-dev/pandas/issues/40686)
!pip install -U sqlalchemy==1.4.46

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.47
    Uninstalling SQLAlchemy-1.4.47:
      Successfully uninstalled SQLAlchemy-1.4.47


In [2]:
import sqlite3

In [3]:
# 데이터베이스 연결
conn = sqlite3.connect('ns_lib.db')

In [4]:
import gdown

gdown.download('https://bit.ly/3RhoNho', 'ns_202104.csv', quiet=False)

Downloading...
From: https://bit.ly/3RhoNho
To: /content/ns_202104.csv
100%|██████████| 57.6M/57.6M [00:01<00:00, 48.0MB/s]


'ns_202104.csv'

**이전에 만든 nslib_book 테이블이 있다면 먼저먼저 삭제해 주세요.**

In [6]:
# DB 커넷견 객체인 conn으로 커서 객체 생성 -> 이 커서 객체의 execute() 메서드에 SQL 문 전달
c = conn.cursor()

c.execute("CREATE TABLE nslib_book \
          (name TEXT, author TEXT, borrow_count INTEGER)")

OperationalError: ignored

In [7]:
# 테이블이 존재하지 않는다면 생성
c.execute("CREATE TABLE IF NOT EXISTS nslib_book \
          (name TEXT, author TEXT, borrow_count INTEGER)")

In [8]:
# 이미 존재하는 테이블 삭제
c.execute("DROP TABLE nslib_book")

In [9]:
c.execute("CREATE TABLE nslib_book \
          (name TEXT, author TEXT, borrow_count INTEGER)")

## 데이터프레임 데이터를 테이블에 추가하기

In [10]:
import pandas as pd

ns_df = pd.read_csv('ns_202104.csv', low_memory=False)
ns_df.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자,Unnamed: 13
0,1,인공지능과 흙,김동훈 지음,민음사,2021,9788937444319,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,2021,9791190123969,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
2,3,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,블랙피쉬,2021,9788968332982,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
3,4,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",문학세계사,2021,9788970759906,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
4,5,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,김영사,2021,9788934990833,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN


In [ ]:
# df 한 행씩 순환
for index, row in ns_df.iterrows():
    c.execute("INSERT INTO nslib_book (name,author,borrow_count) \
              VALUES (?,?,?)", (row['도서명'], row['저자'], row['대출건수']))
    
# 시간이 오래 걸려 비효율적

In [ ]:
for index, row in ns_df.iterrows():
    pass

In [11]:
book_df = ns_df[['도서명','저자','대출건수']]
book_df.head()

,도서명,저자,대출건수
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [12]:
book_df.columns = ['name', 'author', 'borrow_count']
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [13]:
book_df.to_sql('nslib_book', conn, if_exists='replace', index=False)

401682

## 파이썬으로 테이블에서 데이터 읽기

In [14]:
c.execute("SELECT * FROM nslib_book")

In [ ]:
# SELECT 문으로 가져오는 테이블의 첫번째 행 출력
c.fetchone()

('인공지능과 흙', '김동훈 지음', 0)

In [ ]:
c.fetchone()

('가짜 행복 권하는 사회', '김태형 지음', 0)

In [ ]:
# 여러개 출력
c.fetchmany(3)

[('나도 한 문장 잘 쓰면 바랄 게 없겠네', '김선영 지음', 0),
 ('예루살렘 해변', '이도 게펜 지음, 임재희 옮김', 0),
 ('김성곤의 중국한시기행 : 장강·황하 편', '김성곤 지음', 0)]

In [18]:
# 모든행 출력
all_rows = c.fetchall()

In [21]:
book_df = pd.DataFrame(all_rows)
book_df.head()

""


In [20]:
# db를 df로 
book_df = pd.read_sql_query("SELECT * FROM nslib_book", conn)
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [22]:
book_df = pd.read_sql_table('nslib_book', 'sqlite:///ns_lib.db')
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


## 데이터베이스에서 제공하는 함수로 통계량 구하기

In [23]:
len(book_df)

401682

In [24]:
c.execute("SELECT count(*) FROM nslib_book")
c.fetchone()

(401682,)

In [25]:
c.execute("SELECT sum(borrow_count) FROM nslib_book")
c.fetchone()

(4400145,)

In [26]:
c.execute("SELECT avg(borrow_count) FROM nslib_book")
c.fetchone()

(10.95429966988812,)

## 테이블 데이터 정렬하기

In [27]:
c.execute("SELECT * FROM nslib_book ORDER BY borrow_count DESC LIMIT 10")
c.fetchall()

[('사피엔스 :유인원에서 사이보그까지, 인간 역사의 대담하고 위대한 질문 ', '유발 하라리 지음 ;조현욱 옮김', 1468),
 ('해커스 토익:Listening', 'David Cho 지음', 1065),
 ('7년의 밤 :정유정 장편소설 ', '정유정 저', 683),
 ('냉정과 열정사이:Blu', '츠지 히토나리 지음;양억관 옮김', 524),
 ('남한산성:김훈 장편소설', '김훈 지음', 501),
 ('해리포터와 혼혈왕자', '조앤 K. 롤링 지음;최인자 옮김', 451),
 ('해커스 토익:Listening', 'David Cho 지음', 440),
 ('다빈치 코드', '댄 브라운 지음;양선아 옮김', 440),
 ('신:베르나르 베르베르 장편소설', '베르나르 베르베르 지음;이세욱 옮김', 432),
 ('경제학 콘서트', '팀 하포드 지음;김명철 옮김', 425)]

In [28]:
c.close()
conn.close()